# FOSS4G 2025 Demo: Loading Geospatial Data via odc-stac into TerraTorch

This notebook demonstrates how to:
- Connect to STAC catalogs for geospatial data discovery
- Load satellite imagery using odc-stac into xarray Datasets
- Convert data to TerraTorch format for machine learning workflows
- Visualize and explore the loaded data
- Prepare data for geospatial foundation model training

## Overview

**odc-stac** is a powerful library for loading STAC (SpatioTemporal Asset Catalog) items into xarray Datasets, while **TerraTorch** provides tools for fine-tuning geospatial foundation models. This integration enables efficient cloud-native geospatial ML workflows.

**Target Use Cases:**
- Multi-temporal satellite image analysis
- Land cover classification and change detection
- Environmental monitoring and assessment
- Geospatial foundation model fine-tuning

## 1. Install and Import Required Libraries

First, let's install and import all necessary libraries for our geospatial data processing workflow.

In [ ]:
# Install required packages (uncomment if running for the first time)
# !pip install odc-stac terratorch pystac-client xarray rasterio matplotlib cartopy dask

import warnings
warnings.filterwarnings('ignore')

# Core libraries for geospatial data processing
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pathlib import Path

# STAC and data loading libraries
import pystac_client
import odc.stac

# TerraTorch and ML libraries
try:
    import terratorch
    from terratorch.datamodules import GenericNonGeoSegmentationDataModule
    print("TerraTorch imported successfully")
except ImportError:
    print("TerraTorch not available. Install with: pip install terratorch")

# Check library versions
print(f"odc-stac version: {odc.stac.__version__}")
print(f"pystac-client version: {pystac_client.__version__}")
print(f"xarray version: {xr.__version__}")

## 2. Configure STAC Catalog Connection

We'll connect to Element84's Earth Search STAC catalog, which provides free access to a vast collection of geospatial data including Sentinel-2, Landsat, and other Earth observation datasets hosted on AWS Open Data.

In [ ]:
# Configure STAC catalog connection
STAC_URL = "https://earth-search.aws.element84.com/v1"

# Connect to the STAC catalog
catalog = pystac_client.Client.open(STAC_URL)

print(f"Connected to STAC catalog: {STAC_URL}")
print(f"Catalog ID: {catalog.id}")
print(f"Catalog Description: {catalog.description}")

# List available collections
collections = list(catalog.get_collections())
print(f"\nAvailable collections ({len(collections)}):")
for collection in collections[:10]:  # Show first 10
    print(f"  - {collection.id}: {collection.title}")
if len(collections) > 10:
    print(f"  ... and {len(collections) - 10} more collections")

## 3. Search and Discover STAC Items

Now let's search for Sentinel-2 imagery over a specific area of interest. We'll focus on the San Francisco Bay Area for this demonstration.

In [ ]:
# Define area of interest (San Francisco Bay Area)
bbox = [-122.5, 37.4, -121.8, 38.0]  # [min_lon, min_lat, max_lon, max_lat]

# Define time range
start_date = "2023-07-01"
end_date = "2023-07-31"
datetime_range = f"{start_date}/{end_date}"

print(f"Search Parameters:")
print(f"  Area: {bbox}")
print(f"  Time Range: {datetime_range}")
print(f"  Collection: sentinel-2-l2a")

# Search for STAC items
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime=datetime_range,
    query={"eo:cloud_cover": {"lt": 20}}  # Less than 20% cloud cover
)

# Get items from search results
items = list(search.items())
print(f"\nFound {len(items)} Sentinel-2 items")

# Display information about the first few items
for i, item in enumerate(items[:3]):
    print(f"\nItem {i+1}:")
    print(f"  ID: {item.id}")
    print(f"  Date: {item.datetime}")
    print(f"  Cloud Cover: {item.properties.get('eo:cloud_cover', 'N/A')}%")
    print(f"  Assets: {list(item.assets.keys())}")
    print(f"  Geometry: {item.geometry['type']} with {len(item.geometry['coordinates'][0])} points")

## 4. Load Data using odc-stac

Now we'll use odc-stac to load the discovered STAC items into an xarray Dataset. This is where the magic happens - converting cloud-optimized geotiffs into analysis-ready data arrays.

In [ ]:
# Define bands to load (Sentinel-2 10m resolution bands)
bands = ["red", "green", "blue", "nir"]

# Configure loading parameters
load_params = {
    "bands": bands,
    "resolution": 60,  # 60m resolution for faster loading (can be 10, 20, or 60)
    "chunks": {"time": 1, "x": 512, "y": 512},  # Chunking for memory efficiency
    "groupby": "solar_day",  # Group by solar day to handle overlapping scenes
}

print("Loading parameters:")
for key, value in load_params.items():
    print(f"  {key}: {value}")

# Load data using odc-stac
print(f"\nLoading {len(items)} STAC items...")
dataset = odc.stac.load(items, **load_params)

print(f"Successfully loaded dataset!")
print(f"\nDataset Information:")
print(f"  Dimensions: {dict(dataset.dims)}")
print(f"  Coordinates: {list(dataset.coords.keys())}")
print(f"  Data variables: {list(dataset.data_vars.keys())}")
print(f"  Size in memory: {dataset.nbytes / 1e6:.2f} MB")

# Display the dataset
dataset

## 5. Convert to TerraTorch Format

TerraTorch expects data in specific formats. Let's prepare our xarray Dataset for use with TerraTorch's data modules and models.

In [ ]:
def prepare_for_terratorch(dataset: xr.Dataset) -> Dict[str, Any]:
    """
    Prepare xarray Dataset for TerraTorch compatibility.
    
    Args:
        dataset: xarray Dataset loaded from odc-stac
        
    Returns:
        Dictionary with prepared data and metadata
    """
    # Convert to numpy arrays and handle data types
    prepared_data = {}
    
    for band in dataset.data_vars:
        # Convert to float32 for ML compatibility
        data = dataset[band].astype(np.float32)
        
        # Normalize reflectance values (Sentinel-2 values are typically 0-10000)
        data = data / 10000.0
        
        # Clip values to reasonable range
        data = np.clip(data, 0, 1)
        
        prepared_data[band] = data
    
    # Metadata for TerraTorch
    metadata = {
        "bands": list(dataset.data_vars.keys()),
        "spatial_dims": {"x": len(dataset.x), "y": len(dataset.y)},
        "temporal_dim": len(dataset.time) if "time" in dataset.dims else 1,
        "crs": str(dataset.spatial_ref.attrs.get("crs_wkt", "Unknown")),
        "resolution": float(dataset.x[1] - dataset.x[0]) if len(dataset.x) > 1 else 60.0,
        "bbox": [
            float(dataset.x.min()), 
            float(dataset.y.min()),
            float(dataset.x.max()), 
            float(dataset.y.max())
        ],
        "time_range": [
            str(dataset.time.min().values), 
            str(dataset.time.max().values)
        ] if "time" in dataset.dims else None
    }
    
    return {"data": prepared_data, "metadata": metadata}

# Prepare the data
prepared = prepare_for_terratorch(dataset)

print("Data prepared for TerraTorch:")
print(f"  Bands: {prepared['metadata']['bands']}")
print(f"  Spatial dimensions: {prepared['metadata']['spatial_dims']}")
print(f"  Temporal dimension: {prepared['metadata']['temporal_dim']}")
print(f"  CRS: {prepared['metadata']['crs']}")
print(f"  Resolution: {prepared['metadata']['resolution']} meters")
print(f"  Bounding box: {prepared['metadata']['bbox']}")
print(f"  Time range: {prepared['metadata']['time_range']}")

# Sample data shape for first band
sample_band = list(prepared['data'].keys())[0]
print(f"\nSample band '{sample_band}' shape: {prepared['data'][sample_band].shape}")
print(f"Data type: {prepared['data'][sample_band].dtype}")
print(f"Value range: [{prepared['data'][sample_band].min():.3f}, {prepared['data'][sample_band].max():.3f}]")

## 6. Visualize Loaded Data

Let's create some visualizations to verify our data loaded correctly and to explore the temporal and spatial characteristics.

In [ ]:
# Create visualizations
def create_rgb_composite(dataset, time_idx=0, enhance=True):
    """Create an RGB composite image from the dataset."""
    rgb_data = np.stack([
        dataset.red.isel(time=time_idx).values,
        dataset.green.isel(time=time_idx).values,
        dataset.blue.isel(time=time_idx).values
    ], axis=-1)
    
    if enhance:
        # Simple linear stretch enhancement
        rgb_data = np.clip(rgb_data / 3000, 0, 1)  # Sentinel-2 scaling
    
    return rgb_data

# Create figure with subplots
fig = plt.figure(figsize=(16, 12))

# 1. RGB Composite for first time step
ax1 = plt.subplot(2, 3, 1, projection=ccrs.PlateCarree())
rgb_img = create_rgb_composite(dataset, time_idx=0)
extent = [dataset.x.min(), dataset.x.max(), dataset.y.min(), dataset.y.max()]
ax1.imshow(rgb_img, extent=extent, transform=ccrs.PlateCarree())
ax1.add_feature(cfeature.COASTLINE, alpha=0.5)
ax1.add_feature(cfeature.BORDERS, alpha=0.5)
ax1.set_title(f'RGB Composite - {dataset.time[0].dt.strftime("%Y-%m-%d").values}')
ax1.gridlines(draw_labels=True, alpha=0.3)

# 2. NIR band
ax2 = plt.subplot(2, 3, 2)
nir_img = dataset.nir.isel(time=0)
im2 = ax2.imshow(nir_img, extent=extent, cmap='Greens', vmin=0, vmax=3000)
ax2.set_title('Near Infrared (NIR) Band')
ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude')
plt.colorbar(im2, ax=ax2, label='Reflectance')

# 3. NDVI calculation and visualization
ndvi = (dataset.nir - dataset.red) / (dataset.nir + dataset.red)
ax3 = plt.subplot(2, 3, 3)
ndvi_img = ndvi.isel(time=0)
im3 = ax3.imshow(ndvi_img, extent=extent, cmap='RdYlGn', vmin=-0.5, vmax=1.0)
ax3.set_title('NDVI (Vegetation Index)')
ax3.set_xlabel('Longitude')
ax3.set_ylabel('Latitude')
plt.colorbar(im3, ax=ax3, label='NDVI')

# 4. Time series plot for a sample pixel
center_x = len(dataset.x) // 2
center_y = len(dataset.y) // 2
ax4 = plt.subplot(2, 3, 4)

for band in ['red', 'green', 'blue', 'nir']:
    time_series = dataset[band].isel(x=center_x, y=center_y)
    ax4.plot(time_series.time, time_series.values, 'o-', label=band.upper(), alpha=0.8)

ax4.set_title('Temporal Profile (Center Pixel)')
ax4.set_xlabel('Time')
ax4.set_ylabel('Reflectance')
ax4.legend()
ax4.grid(True, alpha=0.3)

# 5. Histogram of NDVI values
ax5 = plt.subplot(2, 3, 5)
ndvi_flat = ndvi.isel(time=0).values.flatten()
ndvi_clean = ndvi_flat[~np.isnan(ndvi_flat)]
ax5.hist(ndvi_clean, bins=50, alpha=0.7, edgecolor='black')
ax5.set_title('NDVI Distribution')
ax5.set_xlabel('NDVI Value')
ax5.set_ylabel('Frequency')
ax5.axvline(ndvi_clean.mean(), color='red', linestyle='--', label=f'Mean: {ndvi_clean.mean():.3f}')
ax5.legend()

# 6. Cloud mask visualization (if available)
ax6 = plt.subplot(2, 3, 6)
if hasattr(dataset, 'scl'):  # Scene Classification Layer
    scl_img = dataset.scl.isel(time=0)
    im6 = ax6.imshow(scl_img, extent=extent, cmap='tab10')
    ax6.set_title('Scene Classification')
    plt.colorbar(im6, ax=ax6, label='Class')
else:
    # Show data coverage
    data_mask = ~np.isnan(dataset.red.isel(time=0).values)
    ax6.imshow(data_mask, extent=extent, cmap='Blues')
    ax6.set_title('Data Coverage')

ax6.set_xlabel('Longitude')
ax6.set_ylabel('Latitude')

plt.tight_layout()
plt.show()

# Print summary statistics
print("\\nSummary Statistics:")
print(f"Dataset temporal coverage: {len(dataset.time)} time steps")
print(f"Spatial coverage: {len(dataset.x)} x {len(dataset.y)} pixels")
print(f"Spatial resolution: ~{abs(dataset.x[1] - dataset.x[0]):.1f} meters")

for band in dataset.data_vars:
    band_data = dataset[band].values
    valid_data = band_data[~np.isnan(band_data)]
    print(f"{band.upper()} band: min={valid_data.min():.0f}, max={valid_data.max():.0f}, mean={valid_data.mean():.0f}")

## 7. Create TerraTorch Dataset

Now let's demonstrate how to create a TerraTorch-compatible dataset from our loaded data. This prepares the data for use with geospatial foundation models.

In [ ]:
# Create a mock TerraTorch dataset class for demonstration
class ODCSTACDataset:
    """
    A demonstration dataset class that wraps odc-stac loaded data
    for use with TerraTorch workflows.
    """
    
    def __init__(self, dataset: xr.Dataset, tile_size: int = 256):
        """
        Initialize the dataset.
        
        Args:
            dataset: xarray Dataset from odc-stac
            tile_size: Size of tiles to extract for training
        """
        self.dataset = dataset
        self.tile_size = tile_size
        self.bands = list(dataset.data_vars.keys())
        
        # Calculate number of possible tiles
        self.tiles_x = (len(dataset.x) - tile_size) // tile_size + 1
        self.tiles_y = (len(dataset.y) - tile_size) // tile_size + 1
        self.n_times = len(dataset.time) if 'time' in dataset.dims else 1
        
        print(f"Dataset initialized:")
        print(f"  Total tiles per timestep: {self.tiles_x * self.tiles_y}")
        print(f"  Time steps: {self.n_times}")
        print(f"  Total samples: {self.tiles_x * self.tiles_y * self.n_times}")
    
    def __len__(self):
        """Return total number of samples."""
        return self.tiles_x * self.tiles_y * self.n_times
    
    def __getitem__(self, idx):
        """
        Get a single sample (tile) from the dataset.
        
        Args:
            idx: Sample index
            
        Returns:
            Dictionary with 'image' and metadata
        """
        # Calculate tile position and time index
        samples_per_time = self.tiles_x * self.tiles_y
        time_idx = idx // samples_per_time
        tile_idx = idx % samples_per_time
        
        tile_y = tile_idx // self.tiles_x
        tile_x = tile_idx % self.tiles_x
        
        # Extract tile coordinates
        x_start = tile_x * self.tile_size
        x_end = x_start + self.tile_size
        y_start = tile_y * self.tile_size
        y_end = y_start + self.tile_size
        
        # Extract data for this tile
        tile_data = {}
        for band in self.bands:
            if 'time' in self.dataset.dims:
                band_data = self.dataset[band].isel(time=time_idx, x=slice(x_start, x_end), y=slice(y_start, y_end))
            else:
                band_data = self.dataset[band].isel(x=slice(x_start, x_end), y=slice(y_start, y_end))
            
            # Convert to numpy and normalize
            band_array = band_data.values.astype(np.float32) / 10000.0  # Normalize Sentinel-2
            tile_data[band] = np.clip(band_array, 0, 1)
        
        # Stack bands into a single array (C, H, W format for PyTorch)
        image = np.stack([tile_data[band] for band in self.bands], axis=0)
        
        # Get coordinates
        x_coords = self.dataset.x.isel(x=slice(x_start, x_end)).values
        y_coords = self.dataset.y.isel(y=slice(y_start, y_end)).values
        
        return {
            'image': image,
            'bands': self.bands,
            'coordinates': {
                'x': x_coords,
                'y': y_coords,
                'time_idx': time_idx
            },
            'metadata': {
                'tile_x': tile_x,
                'tile_y': tile_y,
                'shape': image.shape
            }
        }
    
    def get_sample_batch(self, batch_size: int = 4):
        """Get a batch of samples for demonstration."""
        indices = np.random.choice(len(self), size=batch_size, replace=False)
        batch = [self[idx] for idx in indices]
        return batch

# Create the dataset
odc_dataset = ODCSTACDataset(dataset, tile_size=128)

# Get a sample batch
sample_batch = odc_dataset.get_sample_batch(batch_size=4)

print(f"\nSample batch information:")
for i, sample in enumerate(sample_batch):
    print(f"Sample {i+1}:")
    print(f"  Image shape: {sample['image'].shape}")
    print(f"  Bands: {sample['bands']}")
    print(f"  Tile position: ({sample['metadata']['tile_x']}, {sample['metadata']['tile_y']})")
    print(f"  Value range: [{sample['image'].min():.3f}, {sample['image'].max():.3f}]")

## 8. Demonstrate TerraTorch Functionality

Finally, let's demonstrate some basic TerraTorch-style functionality with our loaded data, including data iteration, batching, and preprocessing transforms.

In [ ]:
# Demonstrate data iteration and preprocessing
def apply_transforms(sample, augment=True):
    """
    Apply preprocessing transforms similar to TerraTorch workflows.
    
    Args:
        sample: Sample dictionary from dataset
        augment: Whether to apply data augmentation
        
    Returns:
        Transformed sample
    """
    image = sample['image'].copy()
    
    if augment:
        # Random horizontal flip
        if np.random.rand() > 0.5:
            image = np.flip(image, axis=2)  # Flip along width
        
        # Random vertical flip
        if np.random.rand() > 0.5:
            image = np.flip(image, axis=1)  # Flip along height
        
        # Small rotation (simplified)
        if np.random.rand() > 0.7:
            # 90-degree rotation
            image = np.rot90(image, axes=(1, 2))
    
    # Normalize to standard range
    image = (image - 0.1) / 0.3  # Example normalization
    image = np.clip(image, -2, 2)
    
    sample_transformed = sample.copy()
    sample_transformed['image'] = image
    sample_transformed['transforms_applied'] = True
    
    return sample_transformed

# Demonstrate batching and preprocessing
print("Demonstrating data iteration and preprocessing:")
print("=" * 50)

# Create multiple batches
for batch_idx in range(3):
    print(f"\nBatch {batch_idx + 1}:")
    
    # Get a batch
    batch = odc_dataset.get_sample_batch(batch_size=2)
    
    # Apply transforms
    transformed_batch = [apply_transforms(sample, augment=True) for sample in batch]
    
    # Display batch statistics
    for i, sample in enumerate(transformed_batch):
        image = sample['image']
        print(f"  Sample {i+1}: shape={image.shape}, range=[{image.min():.2f}, {image.max():.2f}]")
        print(f"    Tile position: ({sample['metadata']['tile_x']}, {sample['metadata']['tile_y']})")

# Visualize some transformed samples
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

sample_batch = odc_dataset.get_sample_batch(batch_size=4)
transformed_batch = [apply_transforms(sample, augment=False) for sample in sample_batch]

for i, (original, transformed) in enumerate(zip(sample_batch, transformed_batch)):
    # Original RGB
    rgb_original = np.transpose(original['image'][:3], (1, 2, 0))  # Convert to HWC
    rgb_original = np.clip(rgb_original, 0, 1)
    axes[0, i].imshow(rgb_original)
    axes[0, i].set_title(f'Original {i+1}')
    axes[0, i].axis('off')
    
    # Transformed RGB (denormalize for display)
    rgb_transformed = np.transpose(transformed['image'][:3], (1, 2, 0))
    rgb_transformed = (rgb_transformed * 0.3) + 0.1  # Reverse normalization
    rgb_transformed = np.clip(rgb_transformed, 0, 1)
    axes[1, i].imshow(rgb_transformed)
    axes[1, i].set_title(f'Transformed {i+1}')
    axes[1, i].axis('off')

plt.tight_layout()
plt.suptitle('Original vs. Transformed Samples', y=1.02, fontsize=14)
plt.show()

# Summary of the complete workflow
print("\n" + "=" * 60)
print("WORKFLOW SUMMARY")
print("=" * 60)
print("✓ Connected to STAC catalog (Planetary Computer)")
print("✓ Searched for Sentinel-2 imagery with cloud filtering")
print("✓ Loaded multi-temporal data using odc-stac into xarray")
print("✓ Converted to TerraTorch-compatible format")
print("✓ Created visualizations and exploratory analysis")
print("✓ Built custom dataset class for ML workflows")
print("✓ Demonstrated data iteration and preprocessing")
print("\nNext steps:")
print("- Integrate with actual TerraTorch models")
print("- Implement custom loss functions for geospatial tasks")
print("- Scale to larger datasets and cloud processing")
print("- Add multi-modal data sources (SAR, hyperspectral)")

## 9. Generate TerraMind Embeddings

Now let's demonstrate generating embeddings using TerraMind, IBM's geospatial foundation model, from our loaded satellite imagery. TerraMind can generate rich 768-dimensional embeddings from 16x16 pixel patches.

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from typing import Tuple, Dict, Any

# TerraMind preprocessing functions
def rgb_smooth_quantiles(rgb_array: np.ndarray) -> np.ndarray:
    """
    Apply smooth quantile normalization to RGB array.
    
    Args:
        rgb_array: RGB array of shape (..., 3)
    
    Returns:
        Normalized RGB array
    """
    # Calculate smooth quantiles (2% and 98%)
    lower_quantile = np.percentile(rgb_array, 2, axis=(0, 1), keepdims=True)
    upper_quantile = np.percentile(rgb_array, 98, axis=(0, 1), keepdims=True)
    
    # Clip and normalize to [0, 1]
    clipped = np.clip(rgb_array, lower_quantile, upper_quantile)
    normalized = (clipped - lower_quantile) / (upper_quantile - lower_quantile + 1e-8)
    
    return normalized

def prepare_terramind_patches(rgb_data: np.ndarray, patch_size: int = 16) -> np.ndarray:
    """
    Prepare RGB data for TerraMind inference by extracting patches.
    
    Args:
        rgb_data: RGB data array of shape (H, W, 3)
        patch_size: Size of square patches (default 16x16)
    
    Returns:
        Array of patches of shape (num_patches, patch_size, patch_size, 3)
    """
    height, width, channels = rgb_data.shape
    
    # Calculate number of patches
    num_patches_h = height // patch_size
    num_patches_w = width // patch_size
    
    # Crop to fit exact patches
    cropped_h = num_patches_h * patch_size
    cropped_w = num_patches_w * patch_size
    cropped_data = rgb_data[:cropped_h, :cropped_w, :]
    
    # Reshape into patches
    patches = cropped_data.reshape(
        num_patches_h, patch_size, num_patches_w, patch_size, channels
    ).transpose(0, 2, 1, 3, 4).reshape(-1, patch_size, patch_size, channels)
    
    return patches

def normalize_terramind_input(patches: np.ndarray) -> torch.Tensor:
    """
    Normalize patches for TerraMind input.
    TerraMind expects inputs normalized with ImageNet statistics.
    
    Args:
        patches: Array of patches of shape (num_patches, 16, 16, 3)
    
    Returns:
        Normalized tensor of shape (num_patches, 3, 16, 16)
    """
    # ImageNet normalization parameters
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    # Convert to tensor and normalize
    patches_tensor = torch.from_numpy(patches).float()
    
    # Transpose to (batch, channels, height, width)
    patches_tensor = patches_tensor.permute(0, 3, 1, 2)
    
    # Normalize
    for i in range(3):
        patches_tensor[:, i] = (patches_tensor[:, i] - mean[i]) / std[i]
    
    return patches_tensor

In [ ]:
# Load TerraMind model
try:
    from terratorch.models.backbones import BACKBONE_REGISTRY
    
    # Initialize TerraMind model
    print("Loading TerraMind model...")
    model = BACKBONE_REGISTRY.build(
        "terramind_v1_base", 
        modalities=["S2RGB"], 
        pretrained=True
    )
    model.eval()
    
    print(f"Model loaded successfully: {type(model)}")
    print(f"Model device: {next(model.parameters()).device}")
    
    # Check if CUDA is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    if device.type == "cuda":
        model = model.to(device)
        print("Model moved to GPU")
    
except ImportError as e:
    print(f"Could not load TerraMind model: {e}")
    print("Please ensure terratorch is installed: pip install terratorch")
    model = None
except Exception as e:
    print(f"Error loading TerraMind model: {e}")
    model = None

In [ ]:
def generate_terramind_embeddings(rgb_data: np.ndarray, model, patch_size: int = 16, batch_size: int = 32) -> Tuple[np.ndarray, Dict[str, Any]]:
    """
    Generate TerraMind embeddings for RGB satellite imagery.
    
    Args:
        rgb_data: RGB data array of shape (H, W, 3), values in [0, 1]
        model: Loaded TerraMind model
        patch_size: Size of patches for processing
        batch_size: Batch size for inference
    
    Returns:
        Tuple of (embeddings array, metadata dict)
    """
    if model is None:
        raise ValueError("Model not loaded. Cannot generate embeddings.")
    
    device = next(model.parameters()).device
    
    # Step 1: Apply smooth quantile normalization
    print("Applying smooth quantile normalization...")
    normalized_rgb = rgb_smooth_quantiles(rgb_data)
    
    # Step 2: Extract patches
    print(f"Extracting {patch_size}x{patch_size} patches...")
    patches = prepare_terramind_patches(normalized_rgb, patch_size)
    print(f"Extracted {len(patches)} patches")
    
    # Step 3: Normalize for model input
    print("Normalizing patches for model input...")
    patches_tensor = normalize_terramind_input(patches)
    
    # Step 4: Generate embeddings in batches
    print("Generating embeddings...")
    embeddings_list = []
    
    with torch.no_grad():
        for i in range(0, len(patches_tensor), batch_size):
            batch = patches_tensor[i:i+batch_size].to(device)
            
            # Generate embeddings
            batch_embeddings = model({"S2RGB": batch})
            
            # Move back to CPU and store
            embeddings_list.append(batch_embeddings.cpu().numpy())
            
            if (i // batch_size + 1) % 10 == 0:
                print(f"Processed {i + len(batch)}/{len(patches_tensor)} patches")
    
    # Combine all embeddings
    embeddings = np.vstack(embeddings_list)
    
    # Create metadata
    metadata = {
        "num_patches": len(patches),
        "patch_size": patch_size,
        "embedding_dim": embeddings.shape[1],
        "original_shape": rgb_data.shape,
        "patches_shape": patches.shape,
        "device_used": str(device)
    }
    
    print(f"Generated {len(embeddings)} embeddings of dimension {embeddings.shape[1]}")
    return embeddings, metadata

In [ ]:
# Generate embeddings from our Auckland data
if model is not None and len(datasets) > 0:
    # Get the most recent image with good quality
    latest_dataset = datasets[-1]  # Most recent
    
    # Create RGB composite
    rgb_array = create_rgb_composite(latest_dataset).values
    
    # Ensure we have valid data
    if not np.isnan(rgb_array).all():
        print(f"Generating TerraMind embeddings for Auckland data...")
        print(f"RGB data shape: {rgb_array.shape}")
        print(f"RGB data range: [{np.nanmin(rgb_array):.3f}, {np.nanmax(rgb_array):.3f}]")
        
        try:
            # Generate embeddings
            embeddings, metadata = generate_terramind_embeddings(
                rgb_array, 
                model, 
                patch_size=16, 
                batch_size=16  # Smaller batch size for safety
            )
            
            print("\nEmbedding Generation Results:")
            print(f"Generated {len(embeddings)} embeddings")
            print(f"Embedding shape: {embeddings.shape}")
            print(f"Embedding statistics:")
            print(f"  Mean: {np.mean(embeddings):.4f}")
            print(f"  Std: {np.std(embeddings):.4f}")
            print(f"  Min: {np.min(embeddings):.4f}")
            print(f"  Max: {np.max(embeddings):.4f}")
            
            # Print metadata
            print(f"\nMetadata:")
            for key, value in metadata.items():
                print(f"  {key}: {value}")
                
            # Store embeddings for later use
            auckland_embeddings = embeddings
            auckland_metadata = metadata
            
        except Exception as e:
            print(f"Error generating embeddings: {e}")
            auckland_embeddings = None
            auckland_metadata = None
    else:
        print("No valid data found for embedding generation")
        auckland_embeddings = None
        auckland_metadata = None
else:
    print("Skipping embedding generation (no model or no data)")
    auckland_embeddings = None
    auckland_metadata = None

In [ ]:
# Visualize and analyze the embeddings
if auckland_embeddings is not None:
    # Create visualization of embedding statistics
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Embedding distribution
    axes[0, 0].hist(auckland_embeddings.flatten(), bins=50, alpha=0.7)
    axes[0, 0].set_title('Distribution of Embedding Values')
    axes[0, 0].set_xlabel('Embedding Value')
    axes[0, 0].set_ylabel('Frequency')
    
    # 2. Mean embedding per patch
    patch_means = np.mean(auckland_embeddings, axis=1)
    axes[0, 1].hist(patch_means, bins=30, alpha=0.7, color='orange')
    axes[0, 1].set_title('Mean Embedding Value per Patch')
    axes[0, 1].set_xlabel('Mean Embedding Value')
    axes[0, 1].set_ylabel('Number of Patches')
    
    # 3. Embedding dimension variance
    dim_variance = np.var(auckland_embeddings, axis=0)
    axes[1, 0].plot(dim_variance)
    axes[1, 0].set_title('Variance Across Embedding Dimensions')
    axes[1, 0].set_xlabel('Embedding Dimension')
    axes[1, 0].set_ylabel('Variance')
    
    # 4. Sample embedding heatmap
    sample_embeddings = auckland_embeddings[:min(50, len(auckland_embeddings))]
    im = axes[1, 1].imshow(sample_embeddings.T, aspect='auto', cmap='viridis')
    axes[1, 1].set_title('Sample Embeddings Heatmap')
    axes[1, 1].set_xlabel('Patch Index')
    axes[1, 1].set_ylabel('Embedding Dimension')
    plt.colorbar(im, ax=axes[1, 1])
    
    plt.tight_layout()
    plt.suptitle('TerraMind Embedding Analysis for Auckland Satellite Data', y=1.02)
    plt.show()
    
    # Calculate some interesting statistics
    print(f"\nEmbedding Analysis:")
    print(f"Total patches processed: {len(auckland_embeddings)}")
    print(f"Embedding dimensionality: {auckland_embeddings.shape[1]}")
    print(f"Most variable dimensions: {np.argsort(dim_variance)[-5:]}")
    print(f"Least variable dimensions: {np.argsort(dim_variance)[:5]}")
    
    # Calculate patch similarity (cosine similarity between first few patches)
    if len(auckland_embeddings) > 1:
        from sklearn.metrics.pairwise import cosine_similarity
        
        # Calculate similarity for first 10 patches
        n_samples = min(10, len(auckland_embeddings))
        similarity_matrix = cosine_similarity(auckland_embeddings[:n_samples])
        
        plt.figure(figsize=(8, 6))
        plt.imshow(similarity_matrix, cmap='coolwarm', vmin=-1, vmax=1)
        plt.colorbar(label='Cosine Similarity')
        plt.title(f'Cosine Similarity Between First {n_samples} Patches')
        plt.xlabel('Patch Index')
        plt.ylabel('Patch Index')
        plt.show()
        
        print(f"Average cosine similarity between patches: {np.mean(similarity_matrix[np.triu_indices_from(similarity_matrix, k=1)]):.3f}")
else:
    print("No embeddings available for visualization")

## 10. Summary and Next Steps

Congratulations! You've successfully completed the full pipeline from STAC data loading to TerraMind embedding generation. Here's what we accomplished:

### What We Built:
1. **STAC Data Loading**: Connected to Microsoft Planetary Computer and loaded Sentinel-2 data for Auckland
2. **Data Preprocessing**: Applied cloud filtering, temporal selection, and band extraction
3. **Visualization**: Created RGB composites, NDVI analysis, and time series plots
4. **TerraTorch Integration**: Prepared data for machine learning workflows
5. **TerraMind Embeddings**: Generated 768-dimensional embeddings from satellite imagery patches

### Key Achievements:
- 🌍 **Global Scale**: Works with any geographic region through STAC catalogs
- ☁️ **Cloud-Native**: Leverages cloud-optimized data formats and catalogs
- 🤖 **Foundation Models**: Integrates state-of-the-art geospatial AI models
- 📊 **Rich Analytics**: Provides comprehensive data analysis and visualization
- 🔄 **Reproducible**: Fully documented workflow with configuration management

### Potential Applications:
- **Change Detection**: Compare embeddings across time to detect changes
- **Land Cover Classification**: Use embeddings as features for ML models
- **Similarity Search**: Find similar geographic regions using embedding similarity
- **Anomaly Detection**: Identify unusual patterns in satellite imagery
- **Multi-temporal Analysis**: Track environmental changes over time

In [ ]:
# Save results for future use
if auckland_embeddings is not None:
    import pickle
    
    # Save complete results
    results = {
        "embeddings": auckland_embeddings,
        "metadata": auckland_metadata,
        "rgb_composite": rgb_array,
        "dataset_info": {
            "dims": dict(datasets[-1].dims),
            "coords": list(datasets[-1].coords.keys()),
            "data_vars": list(datasets[-1].data_vars.keys())
        }
    }
    
    # Save to file
    output_file = "outputs/auckland_terramind_results.pkl"
    with open(output_file, 'wb') as f:
        pickle.dump(results, f)
    
    print(f"Complete results saved to: {output_file}")
    print("You can load these results later with:")
    print(f"with open('{output_file}', 'rb') as f:")
    print("    results = pickle.load(f)")
else:
    print("No results to save (embeddings generation failed)")